# “网络数据采集” 课程

# 《网络数据采集》第六章案例 使用Selenium 爬取去哪儿网信息


## 案例名称

使用Selenium 爬取去哪儿网信息

## 案例内容

本案例使用selenium模拟浏览器登陆，获取翻页操作。

代理可以存入一个文件，程序读取并使用。

支持多进程抓取。

In [12]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-

import os
import time
import datetime
import codecs
import multiprocessing as mp
from os import makedirs
from os.path import exists
from selenium import webdriver
from selenium.webdriver.common.proxy import *
from selenium.webdriver.common.keys import Keys
#导入 ActionChains 类
from selenium.webdriver import ActionChains




def one_driver_ticket(driver, from_city, to_city):
    # time = datetime.datetime.now()
    date = datetime.date.today()
    tomorrow = date+datetime.timedelta(days=1)
    # date格式转为string格式
    tomorrow_string = tomorrow.strftime('%Y-%m-%d')

    driver.find_element_by_xpath('//input[@data-reactid=".0.0.1.0.0.1.0.0.0.0"]').clear() 
    driver.find_element_by_xpath('//input[@data-reactid=".0.0.1.0.0.1.0.0.0.0"]').send_keys(from_city)

    driver.find_element_by_xpath('//input[@data-reactid=".0.0.1.0.0.1.0.2.0.0"]').clear()
    driver.find_element_by_xpath('//input[@data-reactid=".0.0.1.0.0.1.0.2.0.0"]').send_keys(to_city)

    driver.find_element_by_xpath('//input[@data-reactid=".0.0.1.0.0.1.0.3.0.0"]').clear()
    driver.find_element_by_xpath('//input[@data-reactid=".0.0.1.0.0.1.0.3.0.0"]').send_keys(tomorrow_string)

    submitButton = driver.find_element_by_xpath('//button[@data-reactid=".0.0.1.0.0.1.0.9"]')
    
    #submitButton.send_keys(Keys.ENTER)
    ##submitButton.submit()
    
    # 鼠标移动到 ac 位置，在 ac 位置单击
    ActionChains(driver).move_to_element(submitButton).click(submitButton).perform()


    
    time.sleep(10) # 控制间隔时间，等待浏览器反映

    flag = True
    page_num = 0
    while flag:
        # 保存页面
        dstdir = 'ticket/'
        if not exists(dstdir):
            os.mkdir(dstdir)
        filename = from_city + '--'+ to_city + tomorrow_string + str(page_num+1) + '.htm'
        file = os.path.join(dstdir,filename)
        with open(file, 'w',encoding='utf-8') as f:            
            f.write(source_code)
           

        next_page = None
        try:
            next_page = driver.find_element_by_id('nextXI3')
        except Exception as e:
            print(e)
            pass
        print("page: %d" % (page_num+1))
        if next_page:
            try:
                next_page.click()
                time.sleep(2) # 控制间隔时间，等待浏览器反映
                page_num += 1
            except Exception as e:
                print('next_page could not be clicked')
                print(e)
                flag = False
        else:
            flag = False

def get_proxy_list(file_path):
    proxy_list = []
    try:
        f = open(file_path, 'r')
        all_lines = f.readlines() # readlines()每次按行读取整个文件内容，将读取到的内容放到一个列表中，返回list类型。
        for line in all_lines:
            proxy_list.append(line.replace('\r', '').replace('\n', ''))
        f.close()
    except Exception as e:
        print(e)
    return proxy_list

def ticket_worker_proxy(city_proxy):
    city = city_proxy.split(',')[0]
    proxy = city_proxy.split(',')[1]
    proxy = Proxy({
        'proxyType': ProxyType.MANUAL,
        'httpProxy': proxy,
        'ftpProxy': proxy,
        'sslProxy': proxy,
        'noProxy': '' # 过滤不需要代理的地址
    })
    driver = webdriver.Chrome(proxy=proxy)
    driver.get(site)
    driver.maximize_window() # 将浏览器最大化显示
    for i in range(num):
        if city == hot_city_list[i]:
            continue
        from_city = city
        to_city = hot_city_list[i]
        one_driver_ticket(driver, from_city, to_city)
    driver.close()

def all_ticket_proxy():
    hot_city_proxy_list = []
    proxy_list = get_proxy_list('./proxy/proxy.txt') # ./表示当前目录，../表示上一级目录
    for i in range(num):
        hot_city_proxy_list.append(hot_city_list[i]+','+proxy_list[i])
    pool = mp.Pool(processes=1)
    pool.map(ticket_worker_proxy, hot_city_proxy_list) # map(f, [x1, x2, x3, x4]) = [f(x1), f(x2), f(x3), f(x4)]
    pool.close()
    pool.join()

def ticket_worker_no_proxy(city):
    driver = webdriver.Chrome()
    # chromedriver = r'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe'
    # os.environ['webdriver.chrome.driver'] = chromedriver
    # driver = webdriver.Chrome(chromedriver)
    driver.get(site)
    driver.maximize_window() # 将浏览器最大化显示
    time.sleep(5) # 控制间隔时间，等待浏览器反映
    for i in range(num):
        if city == hot_city_list[i]:
            continue
        from_city = city
        to_city = hot_city_list[i]
        one_driver_ticket(driver, from_city, to_city)
    driver.close()
    
def ticket1_no_proxy(from_city,to_city):
    driver = webdriver.Chrome()
    # chromedriver = r'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe'
    # os.environ['webdriver.chrome.driver'] = chromedriver
    # driver = webdriver.Chrome(chromedriver)
    driver.get(site)
    driver.maximize_window() # 将浏览器最大化显示
    time.sleep(5) # 控制间隔时间，等待浏览器反映
    one_driver_ticket(driver, from_city, to_city)
    time.sleep(10)
    driver.close()

def all_ticket_no_proxy():
    
    pool = mp.Pool(processes=1)
    pool.map(ticket_worker_no_proxy, hot_city_list) # map(f, [x1, x2, x3, x4]) = [f(x1), f(x2), f(x3), f(x4)]
    pool.close()
    pool.join()


if __name__ == '__main__':
    print("Start to crawl tickets:")
    start = datetime.datetime.now()
    # all_ticket_proxy() # proxy
    #all_ticket_no_proxy() # no proxy
    ticket1_no_proxy(input("出发地"),input("目的地"))
    end = datetime.datetime.now()
    print("end")
    print("time: ", end - start)

Start to crawl tickets:
出发地北京
目的地海口


NameError: name 'source_code' is not defined

## 案例结论

这个案例应用selenium库实现淘宝商品的搜索和下载。

较为全面地解释了应用selenium的过程。